# Setup

In [4]:
import os
import json
import lancedb
# import dsutils
import numpy as np
import pyarrow as pa
from glob import glob
from tqdm import tqdm
from docarray.typing import NdArray
from typing import Optional
from docarray import BaseDoc, DocList
from FlagEmbedding import BGEM3FlagModel
from docarray.index import HnswDocumentIndex
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

In [5]:
class Document(BaseDoc):
    _id: str
    timestamp: int
    query: str
    title: str
    press: str
    summary: Optional[str] = None
    content: str
    url: str
    origin: str
    embedding: Optional[NdArray] = None

/home/jys/anaconda3/envs/llm/lib/python3.10/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "summary" in "Document" shadows an attribute in parent "BaseDoc"
  warnings.warn(


## MongoDB

In [6]:
client = MongoClient()
hostname = '10.0.10.211'
username = 'root'
password = 'Telepix123#!'
client = MongoClient(hostname, username=username, password=password)
db = client['financial']
news = db.news.find({})

In [7]:
def convert_objectid(doc):
    if "_id" in doc:
        doc["_id"] = str(doc["_id"])
        
    if 'timestamp' in doc:
        doc['timestamp'] = int(doc['timestamp'].timestamp())

    return doc

In [8]:
news = list(news)
news = [convert_objectid(doc) for doc in news]

In [9]:
print(news[0])
print(len(news))

{'_id': '66f27acf577e4ba3072ec717', 'timestamp': 1727165590, 'query': 'BYC', 'title': 'BYC, ‘올데이롱 2024 가을·겨울’ 출시', 'press': '스포츠동아', 'summary': None, 'content': 'BYC가 2030세대를 겨냥한 ‘올데이롱 2024 가을·겨울’(사진)을 내놓았다.눈 내리는 겨울, 따뜻한 오두막에서 느낄 수 있는 아늑함과 편안함을 선사한다. 일상 및 여행 중 가벼운 착용감과 따뜻함을 동시에 느낄 수 있는 실용적이고 편안한 디자인을 추구한다. 캐주얼하게 입을 수 있는 상하 라운지 세트, 겹쳐입기가 가능한 가벼운 이너웨어, 기능성 소재를 활용한 보디히트 등으로 구성했다.', 'url': 'https://n.news.naver.com/mnews/article/382/0001152747?sid=004', 'origin': 'https://sports.donga.com/article/all/20240924/130080665/1'}
1437


## LanceDB

In [10]:
uri = "/media/jys/ssd/workspace/008_PseudoLab/001_Stockelper/server/vectorstore"
db = lancedb.connect(uri)
# async_db = await lancedb.connect_async(uri)

In [11]:
schema = pa.schema([
    pa.field('id', pa.string()),
    pa.field('timestamp', pa.int64()),
    pa.field('query', pa.string()),
    pa.field('title', pa.string()),
    pa.field('press', pa.string()),
    pa.field('summary', pa.string()),
    pa.field('content', pa.string()),
    pa.field('url', pa.string()),
    pa.field('origin', pa.string()),
    pa.field("embedding", pa.list_(pa.float32(), list_size=1024))
])

table = db.create_table("news", schema=schema, mode="overwrite")

In [12]:
table = db.open_table('news')
# async_tbl = async_db.open_table('news')

In [13]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cuda')

Fetching 30 files: 100%|██████████| 30/30 [00:01<00:00, 28.54it/s]


In [14]:
from tqdm.auto import tqdm

docs = []

for data in tqdm(news):
    tmp = 'timestamp: ' + str(datetime.fromtimestamp(data['timestamp'])) + '\n' \
        + 'title: ' + data['title'] + '\n' \
            + 'content: ' + data['content']
    embedding = embedding_function.encode(tmp, return_dense=True, return_sparse=False)
    data['embedding'] = embedding['dense_vecs']
    docs.append(Document(**data))

docs = DocList[Document](docs)
docs = [dict(d) for d in docs]
table.add(docs)

100%|██████████| 1437/1437 [00:42<00:00, 34.04it/s]


In [ ]:
# async_db = await lancedb.connect_async(uri)

## LanceDB Test

In [15]:
table = db.open_table('news')
table.count_rows()
# async_tbl.count_rows()

1437

In [16]:
# table.search("2024-06-10").limit(10).to_list()
table.head()

pyarrow.Table
id: string
timestamp: int64
query: string
title: string
press: string
summary: string
content: string
url: string
origin: string
embedding: fixed_size_list<item: float>[1024]
  child 0, item: float
----
id: [["094281a64037fcf1c8c1b11c660a9974","3a15bd3798b72aac13681bc6bc8a6f49","957d1db41555ffd2305bb7f3d48c2165","25d030521f8e034ff2014e5c212420c0","181e897755b455bf6b9d9f207f1b4b9e"]]
timestamp: [[1727165590,1727166014,1727164028,1727163854,1727164471]]
query: [["BYC","CJ","CJ","DB","GKL"]]
title: [["BYC, ‘올데이롱 2024 가을·겨울’ 출시","CJ 이미경, 亞 여성 기업인 최초 세계시민상 수상","CJ 이미경 부회장, 亞 여성 기업인 최초 세계시민상…“문화 장벽 허물어”","[오픈테크넷2024] AI 시대 최적화된 DB는?...후지쯔 “포스트그레스가 핵심”","GKL, 관광 종사자 역량 강화 아카데미 성료"]]
press: [["스포츠동아","파이낸셜뉴스","국민일보","디지털데일리","일간스포츠"]]
summary: [[null,null,null,null,null]]
content: [["BYC가 2030세대를 겨냥한 ‘올데이롱 2024 가을·겨울’(사진)을 내놓았다.눈 내리는 겨울, 따뜻한 오두막에서 느낄 수 있는 아늑함과 편안함을 선사한다. 일상 및 여행 중 가벼운 착용감과 따뜻함을 동시에 느낄 수 있는 실용적이고 편안한 디자인을 추구한다. 캐주얼하게 입을 수 있는 상하 라운지 세트, 겹쳐입기가 가능한 가벼운 이너웨어, 기능성 소재를 

In [17]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu')

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 377865.23it/s]


In [18]:
from datetime import datetime

query = 'BYC 주식과 관련된 뉴스를 알려줘'
query = embedding_function.encode(query)['dense_vecs']
start_time = '2024-09-23'
start_stamp = datetime.strptime(start_time, "%Y-%m-%d").timestamp()

end_time = '2024-09-25'
end_stamp = datetime.strptime(end_time, "%Y-%m-%d").timestamp()
docs = table.search(query).where(f"(timestamp >= {int(start_stamp)}) AND (timestamp <= {int(end_stamp)})", prefilter=True).limit(5).to_list()
# docs = table.search(query).limit(5).to_list()
# docs = async_tbl.search(query).limit(8).to_list()

In [19]:
datetime.fromtimestamp(docs[0]['timestamp'])

datetime.datetime(2024, 9, 24, 17, 13, 10)

In [ ]:
end_time = '2024-06-11'
end_stamp = datetime.strptime(end_time, "%Y-%m-%d").timestamp()
print(end_stamp)

In [20]:
[(str(datetime.fromtimestamp(doc['timestamp'])), doc['title']) for doc in docs]

[('2024-09-24 17:13:10', 'BYC, ‘올데이롱 2024 가을·겨울’ 출시'),
 ('2024-09-24 18:05:21', '[유가증권시장 공시] JB금융지주 / DN오토모티브'),
 ('2024-09-24 18:05:21', '[유가증권시장 공시] JB금융지주 / DN오토모티브'),
 ('2024-09-24 14:08:09', '[특징주] JB금융지주, ‘주주환원율 50%’ 밸류업 계획에 4%대 오름세'),
 ('2024-09-24 12:30:15', "[직설] 모건스탠리 'K-반도체 겨울' 보고서 내기 직전 SK하이닉스 대량매도")]

In [21]:
sorted([(str(datetime.fromtimestamp(doc['timestamp'])), doc['title']) for doc in docs], key=lambda x : x[0])

[('2024-09-24 12:30:15', "[직설] 모건스탠리 'K-반도체 겨울' 보고서 내기 직전 SK하이닉스 대량매도"),
 ('2024-09-24 14:08:09', '[특징주] JB금융지주, ‘주주환원율 50%’ 밸류업 계획에 4%대 오름세'),
 ('2024-09-24 17:13:10', 'BYC, ‘올데이롱 2024 가을·겨울’ 출시'),
 ('2024-09-24 18:05:21', '[유가증권시장 공시] JB금융지주 / DN오토모티브'),
 ('2024-09-24 18:05:21', '[유가증권시장 공시] JB금융지주 / DN오토모티브')]

In [ ]:
query = '최근 삼성전자의 신제품에 대해 알려줘'
query = embedding_function.encode(query)['dense_vecs']
docs = table.search(query).limit(5).where(f"timestamp > {int(date)}", prefilter=True).to_list()

In [ ]:
sorted([(str(datetime.fromtimestamp(doc['timestamp'])), doc['title']) for doc in docs], key=lambda x : x[0])

In [ ]:
a = "asidhfioas"

a[:12034]